In [1]:
import pandas as pd
import numpy as np  
import sklearn
print (sklearn.__version__)
from scipy import stats
from scipy.stats import  norm,boxcox_normmax
from scipy.stats import skew
from sklearn.model_selection import cross_validate
from scipy.special import boxcox1p
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC
from sklearn.linear_model import Lasso
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LassoCV,ElasticNetCV,RidgeCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
import lightgbm
from sklearn.model_selection import cross_val_score,cross_val_score
from datetime import datetime
from sklearn.model_selection import GridSearchCV    
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import  make_scorer
#from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import warnings
#import shap
#shap.initjs()
warnings.filterwarnings('ignore')

def onehot_enconde(de_in,onehot_list):
    de_out = de_in
    for feature in onehot_list:
        if feature in de_in.columns:
            onehot = pd.get_dummies(de_in[feature],prefix=feature)
            de_out = de_out.join(onehot)
            de_out = de_out.drop(columns = feature)
            print (de_out.shape)
    return de_out


def knn_income(all_data):
    from sklearn import preprocessing
    from sklearn.neighbors import KNeighborsRegressor

    data_train = pd.read_csv("train.csv")
    True_index = data_train['village_income_median'].notnull()
    data_train = data_train [True_index] 

    y_train = data_train['village_income_median']
    x_train = data_train[['lat','lon']]
    model = KNeighborsRegressor(n_neighbors=10, algorithm='brute',weights = 'distance' )
    model.fit(x_train, y_train) 
    return model

#fill miss value // input:tained model , data_all // return fiexed data_all
def fill_miss_value_by_knn(model,data_all):
    False_index =  data_all['village_income_median'].isnull()
    x_test = data_all[False_index]
    x_test = x_test[['lat','lon']]
    if x_test.shape[0]!=0:
        pred = model.predict(x_test)

        income_serise = data_all['village_income_median']
        False_index = income_serise.isnull()
        income_serise [False_index] = pred
        data_all['village_income_median'] = income_serise
    return data_all
def dynamic_selection(number,x_train,y_train):
    onehot_list = ['building_use','building_type','building_material',
                   'town','city','parking_way','village']  

    model = lightgbm.LGBMRegressor(objective='regression',num_leaves=30,
                                   learning_rate=0.1, n_estimators=2000, max_depth=5, bagging_fraction = 0.77,feature_fraction = 0.81,
                                   min_child_samples = 19, min_child_weight = 0.001,
                                   reg_alpha = 0.2, reg_lambda = 0.9,
                                   min_data_in_leaf = 16 , min_split_gain = 0,importance_type='gain',n_job=-1)

    c = [c for c  in x_train.columns if  c in onehot_list ]
    model.fit(x_train,y_train,categorical_feature = c)
    
    index = np.argsort((model.feature_importances_)*-1)
    index = index[0:number]
    cols = list(x_train.columns)
    selected_features = [cols[i] for i in index ]
    
    
          
    return selected_features




def my_custom_loss_func(y_pred, y_true):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    number_of_data = len(y_true)
    diff = np.abs(y_true - y_pred)
    rae = diff/y_true
    z = rae[rae<=0.1]
    hit_ratio = len(z)/number_of_data
    return hit_ratio

def my_custom_loss_func2(model,x, y_true):
    y_pred = model.predict(x)
    y_pred = y_pred #+ x.median_price
    y_true = y_true #+ x.median_price
    number_of_data = len(y_true)
    diff = np.abs(y_true - y_pred)
    rae = diff/y_true
    z = rae[rae<=0.1]
    hit_ratio = len(z)/number_of_data
    return hit_ratio
def fix_parking_area(x_train,x_test):
    x_train = x_train.fillna(0)
    x_test = x_test.fillna(0)
    x_fit = x_train[data_train.parking_area>0]
    y_fit = x_fit.parking_area
    x_fit= x_fit.drop(columns =['total_price','building_id','parking_area'])

    model = lightgbm.LGBMRegressor()
    model.set_params(objective = 'regression', 
                     learning_rate = 0.1,
                     n_estimators = 1000,
                     num_leaves = 100,
                     max_bin = 50,
                     bagging_fraction = 1,
                     bagging_freq = 5,
                     feature_fraction = 1,
                     num_threads = -1)   
    y_fit = np.log1p(y_fit)
    model.fit(x_fit,y_fit)
    pred = model.predict(x_fit)
    pred = np.expm1(pred)
    y_fit = np.expm1(y_fit)
    train_loss = np.sum((np.abs(pred-(y_fit))))/len(y_fit)
    print (train_loss)
    
    train_target_index = (x_train.parking_price>0) & (x_train.parking_area==0)
    test_target_index = (x_test.parking_price>0) & (x_test.parking_area==0)
    temp = x_train[train_target_index].drop(columns =['total_price','building_id','parking_area'])
    pred = model.predict(temp)
    pred = np.expm1(pred)
    pred[pred<1]=1
    x_train['parking_area'][train_target_index] = pred
    
    temp = x_test[test_target_index].drop(columns =['total_price','building_id','parking_area'])
    pred = model.predict(temp)
    pred = np.expm1(pred)
    pred[pred<0]=1
    x_test['parking_area'][test_target_index] = pred
    
    return x_train,x_test

def assign_area_by(k,data_kmeans,data_classifier):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    from sklearn.cluster import KMeans

    scaler = MinMaxScaler()
    data_kmeans.total_price = data_kmeans.total_price/data_kmeans.building_area
    scaled_feature = scaler.fit_transform(data_kmeans[['lat','lon','total_price']])
    KMeans =  KMeans(n_clusters=k, random_state=0).fit(scaled_feature)
    Ground_truth = KMeans.predict(scaled_feature)

    neigh = KNeighborsClassifier(n_neighbors=5)
    scaler2 = MinMaxScaler()
    scaler2.fit(data_classifier)
    neigh.fit(scaler2.transform(data_classifier), Ground_truth)

    return neigh,scaler2


def fill_village(data_x,data_y):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import StandardScaler,MinMaxScaler
    from sklearn.cluster import KMeans

    neigh = KNeighborsClassifier(n_neighbors=10)

    neigh.fit(data_x, data_y)
    

    return neigh




0.21.1


In [2]:
def train_city(i,data_train,data_test,building_type,eval_mode):
    filter = data_train.building_type.isin(building_type)
    data_train = data_train[filter]
    filter = data_test.building_type.isin(building_type)
    data_test=data_test[filter]
    
    
   
    
    
    
    data_train.total_price = data_train.total_price/(data_train.building_area)


    ID_test = pd.DataFrame({'building_id': data_test['building_id'],
                            'building_area': data_test['building_area']})

    all_data = pd.concat([data_train,data_test],keys=['train','test'])
     
        
    
    #x and y
    y_train = all_data['total_price'].loc['train']
    all_data = all_data.drop(columns='total_price')
 
    
    #remove unnecessary featrues
    drop_list = ['building_id',
                 'doc_rate','master_rate','bachelor_rate','jobschool_rate','elementary_rate','highschool_rate', 'junior_rate',
                 'town_population','town_area','town_population_density',
                 'divorce_rate','born_rate','death_rate','marriage_rate']
    
    all_data = all_data.drop(columns=drop_list)



    all_data['lat'] = -all_data['lat']
    all_data['parking_way'] = 2-all_data['parking_way']
    all_data['LA_BA'] =  all_data['land_area']/(all_data['building_area']+1)
    #all_data['LA*BA'] =  all_data['land_area']*(all_data['building_area']+1)
    #all_data['LA+BA'] =  all_data['land_area']+(all_data['building_area']+1)
      
    all_data['PP_BA'] =  all_data['parking_price']/(all_data['building_area']+1)
    all_data['PP_LA'] =  all_data['parking_price']/(all_data['land_area']+1)

    all_data['F_TF'] =  all_data['txn_floor']/(all_data['total_floor'])
    #all_data['F*TF'] =  all_data['txn_floor']*(all_data['total_floor'])
       
    
    all_data['VI_10000/XIII_10000'] =  all_data['VI_10000']/(all_data['XIII_10000'])
    all_data['X_500/XIII_10000'] =  all_data['X_500']/(all_data['XIII_10000'])
    all_data['VI_10000/X_500'] =  all_data['VI_10000']/(all_data['X_500'])
    all_data['II_MIN+I_MIN'] =  all_data['II_MIN']+(all_data['I_MIN'])
    all_data['II_MIN+X_MIN'] =  all_data['II_MIN']+(all_data['X_MIN'])
    all_data['I_MIN+X_MIN'] =  all_data['I_MIN']+(all_data['X_MIN'])
    #all_data['total_MIN'] = all_data.I_MIN+all_data.II_MIN+all_data.III_MIN+all_data.IV_MIN+all_data.V_MIN+all_data.VII_MIN+all_data.VIII_MIN+all_data.XII_MIN
    #all_data['I_MIN+III_MIN'] =  all_data['I_MIN']+(all_data['III_MIN'])
    #all_data['I_MIN+IV_MIN'] =  all_data['I_MIN']+(all_data['IV_MIN'])
    
    #all_data['I_MIN+V_MIN'] =  all_data['I_MIN']+(all_data['V_MIN'])
    
    #all_data['I_MIN+VII_MIN'] =  all_data['I_MIN']+(all_data['VII_MIN'])
    
    #all_data['I_MIN+XII_MIN'] =  all_data['I_MIN']+(all_data['XII_MIN'])
    #I_MIN II_MIN  III_MIN  IV_MIN  V_MIN VII_MIN VIII_MIN  XII_MIN

    all_data['House_Age'] = all_data['txn_dt']-all_data['building_complete_dt']
    all_data['House_Age_0.5y'] = all_data.House_Age.apply(lambda x:(int(x/182)))
    all_data['House_Age_y'] = all_data.House_Age.apply(lambda x:(int(x/365)))
    all_data['House_Age_3y'] = all_data.House_Age.apply(lambda x:(int(x/1095)))

    all_data['txn_dt_0.5y'] =all_data.txn_dt.apply(lambda x:(int(x/182)))
    all_data['txn_dt_y'] =all_data.txn_dt.apply(lambda x:(int(x/365)))
    all_data['txn_dt_3y'] =all_data.txn_dt.apply(lambda x:(int(x/1095)))
    
    all_data['building_complete_dt_0.5y'] =all_data.building_complete_dt.apply(lambda x:(int(x/182)))
    all_data['building_complete_dt_y'] =all_data.building_complete_dt.apply(lambda x:(int(x/365)))
    all_data['building_complete_dt_3y'] =all_data.building_complete_dt.apply(lambda x:(int(x/1095)))

    
    selected_features = dynamic_selection(150,all_data.loc['train'],y_train)
    print (selected_features)
    all_data =all_data[selected_features]   
    
    #onehot encoding
    onehot_list = ['building_use','building_type','building_material',
                   'town','city','parking_way','village']  

    onehot_list = ['building_type','town','city','parking_way']  
    
    cat_index = [all_data.columns.get_loc(c) for c in onehot_list if c in all_data]



    x_train = all_data.loc['train']      
    x_test = all_data.loc['test'] 
    
   

    print("train_x size:",x_train.shape)
    print("train_y size:",y_train.shape)
    print("testing size:",x_test.shape)
    print("ID_test size",ID_test.shape)
    # model 

    if eval_mode:

        from sklearn.preprocessing import QuantileTransformer
        lgb = lightgbm.LGBMRegressor(cat_column = cat_index)
        lgb.set_params(  objective = 'fair', 
                         learning_rate = 0.01,
                         n_estimators = 5000,
                         num_leaves = 250,
                         max_bin = 200,
                         bagging_fraction = 0.8,
                         bagging_freq = 5,
                         feature_fraction = 0.4,
                         num_threads = -1,
                         min_child_samples = 10,)
        #target_trans =  QuantileTransformer(n_quantiles=40000,output_distribution='normal')
        #model = lgb
        model = TransformedTargetRegressor(regressor = lgb, func=np.log1p, inverse_func=np.expm1)
        #model = TransformedTargetRegressor(regressor = lgb,
        #                                 transformer =target_trans )

    
     
    if not eval_mode:
        from sklearn.preprocessing import QuantileTransformer
        lgb = lightgbm.LGBMRegressor(cat_column = cat_index)
        lgb.set_params(  objective = 'fair', 
                         learning_rate = 0.01,
                         n_estimators = 8000,
                         num_leaves = 250,
                         max_bin = 200,
                         bagging_fraction = 0.8,
                         bagging_freq = 5,
                         feature_fraction = 0.4,
                         num_threads = -1,
                         min_child_samples = 10,)
        
        #model = lgb
        
        #target_trans =  QuantileTransformer(n_quantiles=10000,output_distribution='normal')
        #model = lgb
        model = TransformedTargetRegressor(regressor = lgb, func=np.log1p, inverse_func=np.expm1)
        #model = TransformedTargetRegressor(regressor = lgb,
        #                                  transformer =target_trans )

    
    
    eval_score=1
    if eval_mode:
        
        cv_results = cross_validate(model, x_train, y_train,cv=6,
                                    scoring = my_custom_loss_func2,
                                    n_jobs = 1,
                                    return_train_score = True,
                                    verbose = 10 )
        
        

        print ('train score:',np.median(cv_results['train_score']))
        print ('test score:',np.median(cv_results['test_score']))
        eval_score = np.median(cv_results['test_score'])
  
    if not eval_mode:
        model.fit(x_train,y_train)
        pred = model.predict(x_test)
        ID_test['total_price'] = (pred)*(ID_test.building_area)
        print ('train score:',my_custom_loss_func2(model,(x_train),y_train))      
    

    

    return ID_test,ID_test.shape[0],eval_score

In [3]:
result=[]
size = []
score = []
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_test['total_price']=0
data_train.parking_area = data_train.parking_area.fillna(0)
#data_train.parking_price = data_train.parking_price.fillna(0)
#data_train,data_test = fix_parking_area(data_train,data_test)
#model =knn_income(data_train)
#data_train = fill_miss_value_by_knn(model,data_train)
#data_test = fill_miss_value_by_knn(model,data_test)
    

all_data = pd.concat([data_train,data_test],keys=['train','test'])

gp_city = all_data.groupby(['city'])
gp_result = gp_city.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.city = [gp_result.loc[row[1]['city'],'label'] for row in all_data.iterrows() ]

gp_town = all_data.groupby(['city','town'])
gp_result = gp_town.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.town = [gp_result.loc[(row[1]['city'],row[1]['town']),'label'] for row in all_data.iterrows() ]


gp_village = all_data.groupby(['city','town','village'])
gp_result = gp_village.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.village = [gp_result.loc[(row[1]['city'],row[1]['town'],row[1]['village']),'label'] for row in all_data.iterrows() ]

data_train = all_data.loc['train']
data_test = all_data.loc['test']

k=50  
model ,scaler=  assign_area_by(k,data_train[['lat','lon','building_area','total_price']],data_train[['lat','lon']])
data_train['k1'] = model.predict(scaler.transform(data_train[['lat','lon']]))
data_test['k1'] =  model.predict(scaler.transform(data_test[['lat','lon']]))



k=100  
model ,scaler=  assign_area_by(k,data_train[['lat','lon','building_area','total_price']],data_train[['lat','lon']])
data_train['k2'] = model.predict(scaler.transform(data_train[['lat','lon']]))
data_test['k2'] =  model.predict(scaler.transform(data_test[['lat','lon']]))


k=400  
model ,scaler=  assign_area_by(k,data_train[['lat','lon','building_area','total_price']],data_train[['lat','lon']])
data_train['k3'] = model.predict(scaler.transform(data_train[['lat','lon']]))
data_test['k3'] =  model.predict(scaler.transform(data_test[['lat','lon']]))


all_data = pd.concat([data_train,data_test],keys=['train','test'])

gp_city = all_data.groupby(['k1'])
gp_result = gp_city.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.k1 = [gp_result.loc[row[1]['k1'],'label'] for row in all_data.iterrows() ]


gp_city = all_data.groupby(['k2'])
gp_result = gp_city.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.k2 = [gp_result.loc[row[1]['k2'],'label'] for row in all_data.iterrows() ]


gp_city = all_data.groupby(['k3'])
gp_result = gp_city.median().sort_values(by=['parking_price'])
gp_result['label'] = np.arange(len(gp_result))
all_data.k3 = [gp_result.loc[row[1]['k3'],'label'] for row in all_data.iterrows() ]


data_train = all_data.loc['train']
data_test = all_data.loc['test']


In [ ]:

for i in [0]:#np.arange(-1,1,0.01):
    result=[]
    size = []
    score = []




    for building_type in [[0,1,2,3],[4]]:
        print ("training on  :",building_type)
        answer,eval_size,eval_score = train_city(i,data_train,data_test,building_type,1)
        size.append(eval_size)
        score.append(eval_score)
        result.append(answer)


    total_hit_ratio = sum(np.array(size)/(sum(size))*score)
    print ('total_hit_ratio',total_hit_ratio,i)

training on  : [0, 1, 2, 3]
['town', 'k1', 'village', 'k2', 'parking_price', 'XIII_10000', 'building_area', 'XIII_5000', 'building_complete_dt', 'txn_floor', 'txn_dt', 'F_TF', 'k3', 'building_complete_dt_0.5y', 'village_income_median', 'total_floor', 'VI_10000', 'V_MIN', 'X_500', 'land_area', 'VII_500', 'House_Age', 'building_type', 'city', 'II_1000', 'II_MIN', 'IV_5000', 'LA_BA', 'V_500', 'IX_10000', 'I_MIN+X_MIN', 'building_complete_dt_y', 'PP_LA', 'PP_BA', 'V_250', 'X_250', 'VII_250', 'XI_1000', 'II_MIN+I_MIN', 'XIV_500', 'VIII_1000', 'txn_dt_0.5y', 'IX_1000', 'II_MIN+X_MIN', 'parking_area', 'XII_MIN', 'XIII_1000', 'building_complete_dt_3y', 'X_5000', 'I_MIN', 'II_500', 'XIII_MIN', 'V_1000', 'X_500/XIII_10000', 'VIII_500', 'IX_500', 'XIII_500', 'IX_250', 'VIII_MIN', 'XIV_1000', 'VII_MIN', 'House_Age_0.5y', 'IV_1000', 'VII_1000', 'VI_10000/X_500', 'XIV_MIN', 'X_MIN', 'XIV_5000', 'txn_dt_y', 'III_MIN', 'IX_MIN', 'III_500', 'parking_way', 'XII_250', 'VI_MIN', 'I_1000', 'XI_MIN', 'X_100

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ , score=(train=0.997, test=0.654), total= 2.2min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV] ................ , score=(train=0.997, test=0.648), total= 2.2min
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 12.5min remaining:    0.0s


[CV] ................ , score=(train=0.997, test=0.632), total= 2.2min
train score: 0.9968434078334226
test score: 0.6510204081632653
training on  : [4]


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 15.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 15.1min finished


['land_area', 'LA_BA', 'village', 'building_complete_dt', 'House_Age', 'XIII_10000', 'XIV_10000', 'building_area', 'IX_MIN', 'X_1000', 'XIV_MIN', 'III_500', 'VI_MIN', 'XI_250', 'IV_5000', 'lon', 'XII_5000', 'town', 'VII_MIN', 'II_MIN+I_MIN', 'IX_5000', 'k2', 'VII_250', 'X_MIN', 'X_500/XIII_10000', 'city', 'V_250', 'XI_MIN', 'XI_10000', 'I_5000', 'XI_100', 'X_500', 'VI_10000/X_500', 'VII_50', 'IV_250', 'IV_10000', 'I_10000', 'XII_500', 'k1', 'II_5000', 'village_income_median', 'II_1000', 'III_10000', 'VI_5000', 'V_10000', 'XIII_5000', 'IV_MIN', 'VIII_5000', 'V_5000', 'VIII_500', 'k3', 'X_100', 'XII_MIN', 'X_10000', 'XIII_1000', 'XI_5000', 'VIII_1000', 'II_MIN+X_MIN', 'IX_250', 'IV_1000', 'V_1000', 'XII_10000', 'I_MIN', 'House_Age_0.5y', 'V_MIN', 'VI_1000', 'txn_dt', 'VII_10000', 'VIII_MIN', 'X_5000', 'VIII_100', 'VI_500', 'XII_1000', 'VII_1000', 'VI_10000', 'II_500', 'I_MIN+X_MIN', 'VII_500', 'I_250', 'XII_250', 'IX_1000', 'III_1000', 'total_floor', 'VIII_10', 'VII_10', 'III_MIN', 'buil

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
answer = pd.concat(result)
answer = answer.sort_index()
answer = answer.drop(columns = ['building_area'])
answer.to_csv("out.csv", index=False)

In [ ]:
0.6438095238095238